This notebook enables anyone with a Google Colab account to experiment with AI Singapore's SEA-LION models with Ollama. At the time of writing, Google Colab provides a free T4 GPU (16GB GDDR6) runtime.

Please visit https://sea-lion.ai/ for more information about the models.

In [1]:
# Install the dependency
!sudo apt install lshw > apt.log 2>&1 && echo "lshw is installed successfully."

# Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

lshw is installed successfully.
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [7]:
# Start the Ollama service
!ollama serve > ollama.log 2>&1 &

Pull either one or both of the SEA-LION models.

In [3]:
# Pull the Gemma 2 9B CPT SEA-Lionv3 Instruct model
!ollama pull aisingapore/gemma2-9b-cpt-sea-lionv3-instruct:q4_k_m > ollama_pull.log 2>&1 && echo "The model is pulled successfully."

The model is pulled successfully.


In [ ]:
# Pull the Llama 3.1 8B CPT SEA-Lionv3 Instruct model
!ollama pull aisingapore/llama3.1-8b-cpt-sea-lionv3-instruct:q4_k_m > ollama_pull.log 2>&1 && echo "The model is pulled successfully."

The model is pulled successfully.


In [4]:
!ollama list

NAME                                                    ID              SIZE      MODIFIED      
aisingapore/gemma2-9b-cpt-sea-lionv3-instruct:q4_k_m    600c92976f57    5.8 GB    3 seconds ago    


In [10]:
# The simple chat demo is modified from https://github.com/ollama/ollama/blob/main/examples/python-simplechat/client.py
# Please modify it for your use cases.

import json
import requests

# If Ollama is not running, start `ollama serve` in a previous cell
# Use the Gemma 2 9B CPT SEA-Lionv3 Instruct model
model = "aisingapore/gemma2-9b-cpt-sea-lionv3-instruct:q4_k_m"

# Uncomment this statement to use the Llama 3.1 8B CPT SEA-Lionv3 Instruct model
#model = "aisingapore/llama3.1-8b-cpt-sea-lionv3-instruct:q4_k_m"

def chat(messages):
    r = requests.post(
        "http://0.0.0.0:11434/api/chat",
        json={"model": model, "messages": messages, "stream": True},
        stream=True
    )
    r.raise_for_status()
    output = ""

    for line in r.iter_lines():
        body = json.loads(line)
        if "error" in body:
            raise Exception(body["error"])
        if body.get("done") is False:
            message = body.get("message", "")
            content = message.get("content", "")
            output += content
            print(content, end="", flush=True)

        if body.get("done", False):
            message["content"] = output
            return message


def main():
    messages = []
    print ("Input /bye or nothing to exit.")

    while True:
        user_input = input("Enter a prompt: ").strip()
        if not user_input or len(user_input) == 0 or user_input == '/bye':
            print("Exited.")
            break

        print()
        messages.append({"role": "user", "content": user_input})
        message = chat(messages)
        messages.append(message)
        print("\n\n")


if __name__ == "__main__":
    main()

Input /bye or nothing to exit.
Enter a prompt: Translate \"an apple a day keeps the doctor away\" to SEA languages.

Here are translations of "An apple a day keeps the doctor away" into several Southeast Asian languages:

* **Indonesian:** Sebutir apel sehari jauhkan dokter pergi.
* **Malaysian (Bahasa Melayu):** Seekor apel setiap hari menjauhkan doktor.
* **Thai:** แอปเปิลวันละลูก หมอไม่ต้องมา (Apel wan la luk, mao mai dtao ma)
* **Vietnamese:** Một quả táo mỗi ngày giữ bác sĩ xa (Môt quả táo mỗi ngày giữ bác sĩ xa)
* **Filipino:** Isang mansanas araw-araw ay nagpapalayo sa doktor.
* **Myanmar (Burmese):** နေ့စဥ် လတ်သော အပပဲလ တစ်ခု ရသည်မှာ ဆရာဝန်ကို မကူး။ (Nètsù lattasau appeleitkʰo yathāmiṁa sărāwun ko maka)

**Note:** These translations aim for accuracy while preserving the idiomatic meaning of the proverb.  Slight variations might exist depending on regional dialects. 





Enter a prompt: 
Exited.
